# 数据探索

可以不写于实验报告中

# 数据清洗

* 未知值、处理 —————— 提交构建好的新的训练集、测试集

# 模型选择

要求分割训练集为：训练集、验证集

调参

3种分类算法

K种交叉检验

多种评价指标

保存分类结果

模型优化：特征选择、参数调节

# 结果分析

模型性能对比表格、性能对比图

分析性能优劣原因、案例分析



> 提交：数据集、代码、实验分析报告

模型性能图标、性能优化过程及结果的细致分析

## Lab2 Classification

> description here

# 数据探索

阅读 `adult.names` 可以得到完整的对于数据集及任务的描述。

数据集共含有14个属性，1个类别属性，48842条记录（随机按照2:1比例分割，分别存于 `adult.data` 和 `adult.test` 中），3620条记录含有缺失值，处理为"?"。

类别揭示实例的收入，共2类（`>50k`,`<=50k`），比例大致为1:3.

14个属性中，离散的属性为

> 2.workclass;  4.education;  6.marital-status;  7.occupation;  8.relationship;  9.race;  10.sex;   14.native-country.

连续的属性为

> 1.age;  3.fnlwgt;  5.education-num;  11.capital-gain;  12.capital-loss;  13.hours-per-week;

大多数的属性都是字面意思。`fnlwgt` 解释为一种权重值，某些人口统计学特征相近的记录有相近的`fnlwgt`，但是此规律仅在同一州内生效。

## 读取数据

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.latex.repr = False

In [40]:
columns = ['age','workClass','fnlwgt','education','educationNum','martialStatus','occupation','relationship','race','sex','capitalGain','capitalLoss','hoursPerWeek','nativeCountry','income']
train = pd.read_csv('./data/adult.data',header=None,names=columns) # 源数据没有字段行
test = pd.read_csv('./data/adult.test',header=None,names=columns,skiprows=1) # 源数据第一行不是记录
data = pd.concat([train,test])
data.tail(3)

,age,workClass,fnlwgt,education,educationNum,martialStatus,occupation,relationship,race,sex,capitalGain,capitalLoss,hoursPerWeek,nativeCountry,income
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.
16280,35,Self-emp-inc,182148,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,60,United-States,>50K.


使用pandas进行描述性统计：

In [41]:
train.iloc[:,[1,3,5,6,7,8,9,13,14]].describe() # 离散型数据

,workClass,education,martialStatus,occupation,relationship,race,sex,nativeCountry,income
count,32561,32561,32561,32561,32561,32561,32561,32561,32561
unique,9,16,7,15,6,5,2,42,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,22696,10501,14976,4140,13193,27816,21790,29170,24720


In [42]:
test.iloc[:,[1,3,5,6,7,8,9,13,14]].describe() # 离散型数据

,workClass,education,martialStatus,occupation,relationship,race,sex,nativeCountry,income
count,16281,16281,16281,16281,16281,16281,16281,16281,16281
unique,9,16,7,15,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K.
freq,11210,5283,7403,2032,6523,13946,10860,14662,12435


**注意：离散属性的unique值包含了"?"，缺失值**

In [43]:
train.iloc[:,[0,2,4,10,11,12]].describe() #连续型数据

,age,fnlwgt,educationNum,capitalGain,capitalLoss,hoursPerWeek
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [44]:
test.iloc[:,[0,2,4,10,11,12]].describe() #连续型数据

,age,fnlwgt,educationNum,capitalGain,capitalLoss,hoursPerWeek
count,16281.000000,1.628100e+04,16281.000000,16281.000000,16281.000000,16281.000000
mean,38.767459,1.894357e+05,10.072907,1081.905104,87.899269,40.392236
std,13.849187,1.057149e+05,2.567545,7583.935968,403.105286,12.479332
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.167360e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.778310e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.383840e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000


In [46]:
train[train.capitalGain != 0].income.describe()

count      2712
unique        2
top        >50K
freq       1677
Name: income, dtype: object

In [52]:
train[train.capitalLoss != 0].income.describe()

count      1519
unique        2
top        >50K
freq        773
Name: income, dtype: object

要点1：test数据集中，两种标签分别比train多了逗号
> $>50K \rightarrow >50K.$ 

要点2：capitalGain、capitalLoss超过75%的数据同为0. 抽取非0数据后，类别分布与整体差异较大，说明这个属性对收入有一定贡献。但是因为0值过多，可以考虑较少类别的离散化。

In [60]:
data.groupby(by=['fnlwgt']).count().iloc[:,0].sort_values(ascending=False).max()

21

要点3：很多网上的分析把fnlwgt解读为id序号，实际上并不是（因为不唯一）。神奇的fnlwgt值有一定的分布规律。但是考虑数据集的解读，fnlwgt和部分人口统计学特征有重复表达，所以可以考虑去除该属性或者与某些属性进行综合。

In [69]:
data.replace(' ?',np.nan).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48842 entries, 0 to 16280
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            48842 non-null  int64 
 1   workClass      46043 non-null  object
 2   fnlwgt         48842 non-null  int64 
 3   education      48842 non-null  object
 4   educationNum   48842 non-null  int64 
 5   martialStatus  48842 non-null  object
 6   occupation     46033 non-null  object
 7   relationship   48842 non-null  object
 8   race           48842 non-null  object
 9   sex            48842 non-null  object
 10  capitalGain    48842 non-null  int64 
 11  capitalLoss    48842 non-null  int64 
 12  hoursPerWeek   48842 non-null  int64 
 13  nativeCountry  47985 non-null  object
 14  income         48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 6.0+ MB


要点4： 出现缺失值的字段有 `workClass`(2799), `occupation`(2809), `nativeCountry`(857)。这三个变量都是离散型变量，可以考虑使用众数替换的方法填充缺失值。

In [67]:
data[data.workClass==' ?']

,age,workClass,fnlwgt,education,educationNum,martialStatus,occupation,relationship,race,sex,capitalGain,capitalLoss,hoursPerWeek,nativeCountry,income
27,54,?,180211,Some-college,10,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,0,0,60,South,>50K
61,32,?,293936,7th-8th,4,Married-spouse-absent,?,Not-in-family,White,Male,0,0,40,?,<=50K
69,25,?,200681,Some-college,10,Never-married,?,Own-child,White,Male,0,0,40,United-States,<=50K
77,67,?,212759,10th,6,Married-civ-spouse,?,Husband,White,Male,0,0,2,United-States,<=50K
106,17,?,304873,10th,6,Never-married,?,Own-child,White,Female,34095,0,32,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16121,61,?,265201,Some-college,10,Married-civ-spouse,?,Husband,White,Male,0,0,14,United-States,<=50K.
16208,21,?,212661,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
16239,73,?,144872,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,25,Canada,<=50K.
16251,81,?,26711,Assoc-voc,11,Married-civ-spouse,?,Husband,White,Male,2936,0,20,United-States,<=50K.


In [72]:
data.drop_duplicates(subset=['education','educationNum']).iloc[:,3:5].sort_values(by='educationNum')

,education,educationNum
224,Preschool,1
160,1st-4th,2
56,5th-6th,3
15,7th-8th,4
6,9th,5
77,10th,6
3,11th,7
415,12th,8
2,HS-grad,9
10,Some-college,10


整个数据集上， `education` 字段和 `educationNum` 字段都是一一对应的，只有16种组合。

要点5：`education`、`educationNum`字段出现冗余，删除其一。
